## Chapter 20 Non-linear Function Optimization

In [ ]:
from numpy import array, dot, finfo, log, mean, pi, sqrt, squeeze, var, zeros
from numpy.linalg import lstsq
from numpy.random import randn
from scipy.optimize import fmin, fmin_bfgs, fmin_slsqp


# Negative Log-likellihood
def normal_loglik(mu, x):
    n = x.size
    epe = dot(x - mu, x - mu)
    sigma2 = epe / n
    return 0.5 * (n * log(2 * pi) + n * log(sigma2) + epe / sigma2)


# Negative Log-likellihood 2
def normal_loglik2(p, x):
    mu = p[0]
    sigma2 = p[1]
    if sigma2 < 0:
        sigma2 = finfo(float).tiny
    epe = dot(x - mu, x - mu)
    n = x.size
    return 0.5 * (n * log(2 * pi) + n * log(sigma2) + epe / sigma2)


# Negative Log-likellihood 3
def normal_loglik3(p, x):
    mu = p[0]
    sigma2 = p[1] ** 2
    epe = dot(x - mu, x - mu)
    n = x.size
    return 0.5 * (n * log(2 * pi) + n * log(sigma2) + epe / sigma2)


# Negative Log-likellihood 4
def ols_loglik(b, y, X):
    bv = b.view()
    K = X.shape[1]
    bv.shape = K, 1
    e = y - dot(X, bv)
    n = e.size
    epe = dot(e.T, e)
    sigma2 = epe / n
    return squeeze(0.5 * (n * log(2 * pi) + n * log(sigma2) + epe / sigma2))


# Constraint function
def constraint(p, x):
    return array([1, p[1] - 0.0001])

## Exercise 1    


In [ ]:
x = randn(1000) + 2
mu0 = mean(x) + 1
args = (x,)
mu = fmin_bfgs(normal_loglik, mu0, args=args)
print("mu:")
print(mu)
print("mean(x):")
print(mean(x))
mu = fmin(normal_loglik, mu0, args=args)
print("mu:")
print(mu)

## Exercise 2


In [ ]:
p0 = zeros(2)
p0[0] = mean(x) + 1
p0[1] = var(x) + 2
print("normal_loglik2(p0,x):")
print(normal_loglik2(p0, x))
p = fmin_slsqp(normal_loglik2, p0, args=args, bounds=[(-100, 100), (0.000001, 100)])
print("array([mean(x),var(x)]):")
print(array([mean(x), var(x)]))
print("p:")
print(p)
p = fmin_slsqp(normal_loglik2, p0, args=args, f_ieqcons=constraint)
print("p:")
print(p)

## Exercise 3


In [ ]:
from scipy.optimize import minimize

args = (x,)
p = minimize(normal_loglik2, p0, args=args, bounds=[(-100, 100), (0.000001, 100)])
print("array([mean(x),var(x)]):")
print(array([mean(x), var(x)]))
print("p:")
print(p)
minimize_cons = {"type": "ineq", "fun": lambda p: p[1] - 0.0001}
p = minimize(normal_loglik2, p0, args=args, constraints=minimize_cons)
print("p:")
print(p)

## Exercise 4


In [ ]:
p0[1] = sqrt(p0[1])
p = fmin_bfgs(normal_loglik3, p0, args=args)
p[1] = p[1] ** 2
print("p:")
print(p)

## Exercise 5


In [ ]:
b = array([[1.0], [2]])
X = randn(1000, 2)
y = dot(X, b) + randn(1000, 1)
b0 = lstsq(X, y, rcond=None)
b0 = b0[0]
print("b0:")
print(b0)
args = (y, X)
print("ols_loglik(b0.ravel(),y,X):")
print(ols_loglik(b0.ravel(), y, X))
b = fmin_slsqp(ols_loglik, b0.ravel(), args=args)
print("b:")
print(b)